In [7]:
audiobook = {
    "total_time": 25383.568005,
    "content_start": 11.563,
    "content_end": 25345.675,
    "current_time": 688.886,
    "current_chapter": 2,
    "chapter_start": 576.365,
    "chapter_end": 1109.053,
}

ebook = {
    
}

In [5]:
688.886-576.365

112.52099999999996

In [8]:
def get_audio_percentage(a):
    content_time = a["content_end"] - a["content_start"]
    current_percentage = (a["current_time"] - a["content_start"]) / content_time
    chapter_percentage = (a["current_time"] - a["chapter_start"]) / (a["chapter_end"] - a["chapter_start"])
    return current_percentage, chapter_percentage

get_audio_percentage(audiobook)

(0.02673561244222809, 0.21123246628420378)

In [16]:
content_keywords = [
    "part",
    "chapter",
    "epilogue",
    "prologue"
]

# ascending numbers. anything inbetween is a chapter

discard_keywords = [
    "copyright",
    "dedication",
    "acknowledgements",
    "contents",
    "table of contents",
    "about",
    "cover",
    "epigraph",
    "note",
    "map",   
    "list",
    "postscript"
]

In [41]:
from ebooklib import epub
import ebooklib

# Load the EPUB file
book = epub.read_epub('data/creative.epub')

# Extract metadata
toc = book.toc
for element in toc:
    if isinstance(element, ebooklib.epub.Link):
        print(element.title, element.href)
    elif isinstance(element, tuple):
        print(element[0].title, element[0].href)


Cover OEBPS/xhtml/001_Cover.xhtml
Title Page OEBPS/xhtml/002_Title_Page.xhtml
Copyright OEBPS/xhtml/003_Copyright.xhtml
Epigraph OEBPS/xhtml/004_Epigraph.xhtml
78 Areas of Thought OEBPS/xhtml/005_78_Areas_of_Thought.xhtml
Note to Reader OEBPS/xhtml/006_Poem.xhtml
Everyone Is a Creator OEBPS/xhtml/007_Everyone_Is_a_Creato.xhtml
Tuning In OEBPS/xhtml/008_Tuning_In.xhtml
The Source of Creativity OEBPS/xhtml/010_The_Source_of_Creati.xhtml
Awareness OEBPS/xhtml/012_Awareness.xhtml
The Vessel and the Filter OEBPS/xhtml/014_The_Vessel_and_the_F.xhtml
The Unseen OEBPS/xhtml/016_The_Unseen.xhtml
Look for Clues OEBPS/xhtml/018_Look_for_Clues.xhtml
Practice OEBPS/xhtml/020_Practice.xhtml
Submerge (The Great Works) OEBPS/xhtml/022_Submerge_The_Great_W.xhtml
Nature as Teacher OEBPS/xhtml/023_Nature_as_Teacher.xhtml
Nothing Is Static OEBPS/xhtml/025_Nothing_Is_Static.xhtml
Look Inward OEBPS/xhtml/027_Look_Inward.xhtml
Memories and the Subconscious OEBPS/xhtml/028_Memories_and_the_Sub.xhtml
It’s Alwa

c:\Users\dangy\Documents\git\notebooks\.venv\Lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
c:\Users\dangy\Documents\git\notebooks\.venv\Lib\site-packages\ebooklib\epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [46]:
import os
from ebooklib import epub
import re

# Keywords for filtering
CONTENT_KEYWORDS = [
    "part", "chapter", "epilogue", "prologue"
]
EXCLUDE_KEYWORDS = [
    "copyright", "dedication", "acknowledgements", "contents", 
    "table of contents", "about", "cover", "epigraph", 
    "note", "map", "list", "postscript", "cover"
]


def classify_titles(toc_titles):
    classified = []
    for title in toc_titles:
        title_lower = title.lower()
        
        # Rule 1: Contains content keywords
        if any(keyword in title_lower for keyword in CONTENT_KEYWORDS):
            classified.append((title, "main"))
            continue
        
        # Rule 3: Ascending numbering (Roman numerals, Arabic, or written numbers)
        if re.search(r'\b(?:chapter|part|)\s*(\b[ivxlcdm]+\b|\d+|one|two|three|four|five|six|seven|eight|nine|ten)\b', title_lower):
            classified.append((title, "main"))
            continue
        
        # Rule 4: Exclude keywords
        if any(keyword in title_lower for keyword in EXCLUDE_KEYWORDS):
            classified.append((title, "not"))
            continue
        
        # Default to not_content for titles not matching any rule
        classified.append((title, "not"))
    
    return classified

def apply_between_rule(classified):
    # Adjust titles between two "main" entries
    for i in range(1, len(classified) - 1):
        current = classified[i]
        previous = classified[i - 1]
        next_title = classified[i + 1]
        
        if (
            current[1] == "not"
            and previous[1] == "main"
            and next_title[1] == "main"
        ):
            # Mark the current title as main
            classified[i] = (current[0], "main")
    
    return classified

def filter_toc(toc):
    # Extract titles from the TOC
    toc_titles = [item.title if isinstance(item, epub.Link) else item[0].title for item in toc]
    
    # Step 1: Initial classification without "between rule"
    classified = classify_titles(toc_titles)
    
    # Step 2: Apply "between two main content" rule
    classified = apply_between_rule(classified)
    
    return classified

def process_epub_files(directory):
    # Walk through all subdirectories and files
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.epub'):
                epub_path = os.path.join(root, file)
                print(f"Processing: {epub_path}")
                
                try:
                    book = epub.read_epub(epub_path)
                    filtered_toc = filter_toc(book.toc)
                    
                    # Write results to a file
                    output_path = os.path.join(root, f"{os.path.splitext(file)[0]}_filtered_toc.txt")
                    with open(output_path, 'w', encoding='utf-8') as f:
                        for title, category in filtered_toc:
                            f.write(f"{title}, {category}\n")
                    
                    print(f"Results written to: {output_path}")
                
                except Exception as e:
                    print(f"Error processing {epub_path}: {e}")

# Run the script on all .epub files in the 'data' folder
process_epub_files(r'Z:\books\calibre-library')


Processing: Z:\books\calibre-library\Alexandra Benedict\Murder on the Christmas Express (237)\Murder on the Christmas Express - Alexandra Benedict.epub
Error processing Z:\books\calibre-library\Alexandra Benedict\Murder on the Christmas Express (237)\Murder on the Christmas Express - Alexandra Benedict.epub: [Errno 13] Permission denied: 'Z:\\books\\calibre-library\\Alexandra Benedict\\Murder on the Christmas Express (237)\\Murder on the Christmas Express - Alexandra Benedict_filtered_toc.txt'
Processing: Z:\books\calibre-library\Alexandra Benedict\The Christmas Jigsaw Murders_ The new deliciously dark Christmas cracker from the bestselling a (264)\The Christmas Jigsaw Murders_ The new deli - Alexandra Benedict.epub
Error processing Z:\books\calibre-library\Alexandra Benedict\The Christmas Jigsaw Murders_ The new deliciously dark Christmas cracker from the bestselling a (264)\The Christmas Jigsaw Murders_ The new deli - Alexandra Benedict.epub: [Errno 13] Permission denied: 'Z:\\books\

KeyboardInterrupt: 